In [1]:
import os
import re
import gzip
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
def make_content(file):
    with gzip.open(file, "rb") as f:
        data = f.readlines()
        data = [d.decode() for d in data]
        data = "".join(data)
    bs_data = bs(data, "lxml")
    sentences = bs_data.find_all("se")
    return sentences

In [3]:
def get_data(file, content_arr, idx):
    sentences = make_content(file)
    for _, sent in enumerate(sentences):
        sent_arr = []
        sent_arr.append(str(idx))
        lex_sent = []
        full_sent = []
        for word in sent.find_all('w'):
            ana = word.find('ana')
            lemma = ana['lex']
            try:
                gram = ana['gr'].split(',')[0]
                result = lemma + '_' + gram
            except KeyError:
                print(word)
                result = lemma
            lex_sent.append(result)
        lex_str = ' '.join(lex_sent)
        sent_arr.append(lex_str)        
        full_sent = sent.text.strip().replace("\n", " ")
        sent_arr.append(full_sent)
        content_arr.append(sent_arr)
        idx += 1
    return idx

In [ ]:
def write_gzip(file, root):
    in_path =  root + '\\' + file
    out_path = root[:32] + '_gzip' + root[32:]
    try:
        os.makedirs(fr'{out_path}')
    except FileExistsError:
        pass
    with open(in_path, 'rb') as f_in, gzip.open(out_path + '\\' + file + '.gz', 'wb') as f_out:
        f_out.writelines(f_in)

In [ ]:
def process_file(file, root):
    in_path = root + '\\' + file
    out_path = root[:32] + '_no_markup' + root[32:]
    try:
        os.makedirs(fr'{out_path}')
    except FileExistsError:
        pass
    with open(in_path, "r", encoding="utf-8") as f:
        data = f.read()
    bs_data = BeautifulSoup(data, "xml")
    sentences = bs_data.find_all("se")
    with open(out_path + '\\' + file[:-3] + 'txt', "w", encoding="utf-8") as f:
        my_text = ""
        for sentence in sentences:
            for w in sentence.find_all("w"):
                my_text += process_word(w)
            my_text += "\n"
        f.write(my_text)

In [12]:
count = 0
idx = 0
content_arr = []
for root, dirs, files in tqdm(os.walk(r"E:/DATA/nkrya_full_source_merged_gzip/")):
    for file in tqdm(files):
        if not file.startswith(".") and file.endswith("xml.gz"):
            in_path = root + '\\' + file
#             write_gzip(file, root) # archive files (gzip)
#             process_file(file, root) # write to word_LEMMA format
            idx = get_data(in_path, content_arr, idx)
            count += 1

In [220]:
df = pd.DataFrame(content_arr, columns = ['ID','LEMMAS', 'RAW'])
df.to_csv("corpus_bigger.csv.gz", encoding = 'utf-8', index = False, compression="gzip", line_terminator="\n)